In [0]:
a=[]
while (1):
  a.append('1')

## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
root_path = '/content/drive/My Drive/AIML/DogBreed_Classification'  #change dir to your project folder

In [3]:
ls

drive/  labels.csv  sample_data/  sample_submission.csv  test/  train/


In [4]:
import os
import time
from __future__ import absolute_import
from __future__ import print_function
from datetime import timedelta
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import cv2

%matplotlib inline
plt.rcParams['figure.figsize'] = (16.0, 4.0) # Set default figure size

Using TensorFlow backend.


In [0]:
TENSORBOARD_SUMMARIES_DIR = '/tmp/DogBreeds_classifier_logs'

In [0]:
from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/AIML/DogBreed_Classification/'+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/AIML/DogBreed_Classification/'+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/AIML/DogBreed_Classification/'+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/AIML/DogBreed_Classification/'+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
import pandas as pd
db_labels=pd.read_csv('labels.csv')

### Print the count of each category of Dogs given in the dataset



In [11]:
db_labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [12]:
db_labels.count()

id       10222
breed    10222
dtype: int64

In [13]:
db_labels_x=db_labels['breed'].value_counts()
db_labels_x.head(10)

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
shih-tzu                112
pomeranian              111
great_pyrenees          111
basenji                 110
samoyed                 109
Name: breed, dtype: int64

### Get one-hot encodings of labels

In [0]:
import tensorflow as tf
from sklearn import preprocessing

In [0]:
y_target=pd.Series(db_labels['breed'])

In [0]:
one_hot=pd.get_dummies(y_target,sparse=True)
y_target=np.asarray(one_hot)

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
img_rows = 128
img_cols = 128
num_channel = 3

In [0]:

from tqdm import tqdm
ids=db_labels['id']
x_train=[]
y_train=y_target
#print(y_train[1].shape)

for id in ids:
    img = cv2.imread('./train/{}.jpg'.format(id))
    x_train.append(cv2.resize(img,(img_rows,img_cols)))
    
x_train=np.array(x_train,np.float32)

In [19]:
x_train.shape

(10222, 128, 128, 3)

In [20]:
y_train.shape

(10222, 120)

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [21]:
import numpy as np

x_train = (np.array(x_train, np.float32) /255)   # /= 255 for normolisation
print (x_train.shape)

#y_train = np.array(db_labels)

#print(y_train.shape)

(10222, 128, 128, 3)


### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.30, random_state=1)

In [23]:
print (x_train.shape)
print (y_train.shape)
print (x_test.shape)
print (y_test.shape)

(7155, 128, 128, 3)
(7155, 120)
(3067, 128, 128, 3)
(3067, 120)


### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
sample_s=pd.read_csv('sample_submission.csv')
test_img=sample_s['id']

In [25]:
test_img[0]

'000621fb3cbb32d8935728e48679680e'

Run the below code to load the test image files in x_test_feature

In [26]:
img_rows=128
img_cols=128
x_test_feature = []
i = 0 # initialization
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f))
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_test_feature.append(img_resize)


100%|██████████| 10357/10357 [00:31<00:00, 328.00it/s]


Normalize the test data and convert it into 4 dimensions

In [27]:
x_test_feature=np.array(x_test_feature,np.float32)

x_test_feature.shape

(10357, 128, 128, 3)

In [0]:
x_test_feature=x_test_feature/255

In [29]:
print (x_train.shape,x_test.shape)

(7155, 128, 128, 3) (3067, 128, 128, 3)


In [30]:
print(x_test_feature.shape)

(10357, 128, 128, 3)


### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
from keras.models import Sequential
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalMaxPooling2D,Activation
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import MaxPooling2D

In [0]:
num_classes=120
batch_size=128
epochs=10

In [34]:
model=Sequential()
model.add(Conv2D(32,(5,5),input_shape=x_train.shape[1:],activation='relu',padding='same'))
model.add(BatchNormalization(axis=3))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))

model.add(Conv2D(32,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(120))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      2432      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)            

### Use batch_size = 128 and epochs = 10 and execute the model

In [36]:
print(x_train.shape)
print(y_train.shape)

(7155, 128, 128, 3)
(7155, 120)


In [37]:
model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
7155/7155 [==============================] - 10s 1ms/step - loss: 5.6174 - acc: 0.0095
Epoch 2/10
7155/7155 [==============================] - 3s 411us/step - loss: 4.7816 - acc: 0.0130
Epoch 3/10
7155/7155 [==============================] - 3s 412us/step - loss: 4.7678 - acc: 0.0168
Epoch 4/10
7155/7155 [==============================] - 3s 410us/step - loss: 4.7530 - acc: 0.0183
Epoch 5/10
7155/7155 [==============================] - 3s 416us/step - loss: 4.7187 - acc: 0.0189
Epoch 6/10
7155/7155 [==============================] - 3s 407us/step - loss: 4.6824 - acc: 0.0268
Epoch 7/10
7155/7155 [==============================] - 3s 407us/step - loss: 4.6174 - acc: 0.0348
Epoch 8/10
7155/7155 [==============================] - 3s 404us/step - loss: 4.5435 - acc: 0.0401
Epoch 9/10
7155/7155 [==============================] - 3s 403us/step - loss: 4.4639 - acc: 0.0486
Epoch 10/10
71

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
from keras.preprocessing.image import ImageDataGenerator,img_to_array, load_img

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=False, 
    samplewise_center=False, 
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [40]:
y_test.shape

(3067, 120)

In [0]:
train_generator = datagen.flow(x_train, y_train, batch_size=50)

In [0]:
val_generator = datagen.flow(x_test, y_test, batch_size=50)

In [44]:
tf.keras.backend.clear_session()

from keras.models import Sequential

from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout

#Initialize model, reshape & normalize data
model = tf.keras.models.Sequential()

#Add first convolutional layer
model.add(tf.keras.layers.Conv2D(50, #Number of filters 
                                 kernel_size=(5,5), #Size of the filter
                                 activation='relu'
                                , input_shape=(128,128,3)))



#Add second  convolutional layer
model.add(tf.keras.layers.Conv2D(50, #Number of filters 
                                 kernel_size=(3,3), #Size of the filter
                                 activation='relu'
                                , input_shape=(128,128,3)))

model.add(tf.keras.layers.Dropout(0.25))

#Flatten the output
model.add(tf.keras.layers.Flatten())

#Dense layer
model.add(tf.keras.layers.Dense(256, activation='relu'))

#Dense layer
model.add(tf.keras.layers.Dense(120, activation='softmax'))


model.compile(optimizer='adam', 
              loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 124, 50)      3800      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 50)      22550     
_________________________________________________________________
dropout (Dropout)            (None, 122, 122, 50)      0         
_________________________________________________________________
flatten (Flatten)            (None, 744200)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               190515456 
_________________________________________________________________
dense_1 (Dense)              (None, 120)               30840     
Total params: 190,572,646
Trainable param

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [45]:
x_train.shape

(7155, 128, 128, 3)

In [46]:
# Set up the generator
#Train the model
history = model.fit_generator(train_generator, 
                             validation_data=val_generator , use_multiprocessing=True,
                     steps_per_epoch=len(x_train/50), epochs=10)

Epoch 1/10
7155/7155 [==============================] - 1498s 209ms/step - loss: 3.8575 - acc: 0.1330 - val_loss: 5.3490 - val_acc: 0.0597
Epoch 2/10
7155/7155 [==============================] - 1479s 207ms/step - loss: 2.2599 - acc: 0.4286 - val_loss: 7.2782 - val_acc: 0.0734
Epoch 3/10
7155/7155 [==============================] - 1517s 212ms/step - loss: 1.5213 - acc: 0.5997 - val_loss: 8.1973 - val_acc: 0.0694
Epoch 4/10
7155/7155 [==============================] - 1509s 211ms/step - loss: 1.1712 - acc: 0.6876 - val_loss: 9.3299 - val_acc: 0.0714
Epoch 5/10
7155/7155 [==============================] - 1519s 212ms/step - loss: 0.9733 - acc: 0.7381 - val_loss: 10.4336 - val_acc: 0.0740
Epoch 6/10
7155/7155 [==============================] - 1514s 212ms/step - loss: 0.8475 - acc: 0.7721 - val_loss: 11.2301 - val_acc: 0.0691
Epoch 7/10
7155/7155 [==============================] - 1514s 212ms/step - loss: 0.7661 - acc: 0.7945 - val_loss: 11.9947 - val_acc: 0.0756
Epoch 8/10
7155/7155 [==

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input

In [0]:
from keras import backend as K

In [49]:
K.clear_session()

In [0]:
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=('/content/drive/My Drive/vgg16_weights_tf_dim_ordering_tf_kernels_notop1.h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

In [51]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

### Make all the layers in the base_model (VGG16) to be non-trainable

In [52]:

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout 
from keras.layers.normalization import BatchNormalization


model_transfer = Sequential()
model_transfer.add(Dense(1024,input_dim=512, activation='relu'))
model_transfer.add(BatchNormalization())
model_transfer.add(Dropout(0.50))
model_transfer.add(Dense(256, activation='relu'))
model_transfer.add(BatchNormalization())
model_transfer.add(Dropout(0.40))
model_transfer.add(Dense(256, activation='relu'))
model_transfer.add(BatchNormalization())
model_transfer.add(Dropout(0.30))

model_transfer.add(Dense(120,activation='softmax'))
model_transfer.compile(optimizer='adam', 
              loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [53]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in base_model.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in base_model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

input_1
False
block1_conv1
False
block1_conv2
False
block1_pool
False
block2_conv1
False
block2_conv2
False
block2_pool
False
block3_conv1
False
block3_conv2
False
block3_conv3
False
block3_pool
False
block4_conv1
False
block4_conv2
False
block4_conv3
False
block4_pool
False
block5_conv1
False
block5_conv2
False
block5_conv3
False
block5_pool
False
global_average_pooling2d_1
False


### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [0]:

x_train = base_model.predict(x_train)

x_test = base_model.predict(x_test)

In [55]:
output_transfer = model_transfer.fit(x_train, y_train, 
                                      validation_data=(x_test, y_test),
                                      batch_size=300, epochs=150, verbose=1)

Train on 7155 samples, validate on 3067 samples
Epoch 1/150
7155/7155 [==============================] - 1s 151us/step - loss: 5.4284 - acc: 0.0131 - val_loss: 4.6107 - val_acc: 0.0437
Epoch 2/150
7155/7155 [==============================] - 0s 40us/step - loss: 4.7208 - acc: 0.0447 - val_loss: 4.2460 - val_acc: 0.0783
Epoch 3/150
7155/7155 [==============================] - 0s 38us/step - loss: 4.3487 - acc: 0.0791 - val_loss: 3.9991 - val_acc: 0.1043
Epoch 4/150
7155/7155 [==============================] - 0s 39us/step - loss: 4.0111 - acc: 0.1058 - val_loss: 3.8143 - val_acc: 0.1311
Epoch 5/150
7155/7155 [==============================] - 0s 40us/step - loss: 3.7935 - acc: 0.1346 - val_loss: 3.6890 - val_acc: 0.1448
Epoch 6/150
7155/7155 [==============================] - 0s 39us/step - loss: 3.5702 - acc: 0.1597 - val_loss: 3.5486 - val_acc: 0.1722
Epoch 7/150
7155/7155 [==============================] - 0s 37us/step - loss: 3.4376 - acc: 0.1855 - val_loss: 3.4645 - val_acc: 0.1845